In [112]:
from IPython.display import HTML

"""Cash ladder

Demonstration of how to compute a cash ladder for a portfolio.

Attributes
----------
cash
"""

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [113]:
import warnings
warnings.filterwarnings('ignore')

In [114]:
# Import general purpose packages
import os
import json
from datetime import datetime, timedelta
import pytz
#from get_cashforecast import cash_forecast

# Import lusid specific packages
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id

# Import data wrangling packages
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")
print(secrets_path)

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

/Users/peternagymathe/secrets/fbn-ci.json


In [115]:
## DATA
with open("config/cash_ladder_seed_data.json", "r") as seed_data_config:
    mapping_file = json.loads(seed_data_config.read())
    
# Create a new scope

scope = "cash-ladder-001"
portfolio_code = "EQUITY_FUND"

# Load a file of equity transactions

transactions_file = r"data/cash_ladder.csv"
transactions_df = pd.read_csv(transactions_file)
transactions_df["portfolio_code"] = portfolio_code

# Load portfolios, instruments, and transactions

seed_data_response = seed_data(
    api_factory,
    ["portfolios", "instruments", "transactions"],
    scope,
    transactions_file,
    "csv",
    mappings = mapping_file
)

In [116]:
from cashladder import get_cash_ladder
from datetime import timezone
start_date = datetime(2020, 4, 1, tzinfo=pytz.UTC)
print(start_date)
cladder = get_cash_ladder(start_date, portfolio_code, "GBP", scope)
print(cladder)

2020-04-01 00:00:00+00:00
   ReportRunTime SettlementDate Currency    Portfolio  \
0       11:49:44     2020-04-01      GBP  EQUITY_FUND   
1       11:49:44     2020-04-02      GBP  EQUITY_FUND   
2       11:49:44     2020-04-02      GBP  EQUITY_FUND   
3       11:49:44     2020-04-03      GBP  EQUITY_FUND   
4       11:49:44     2020-04-03      GBP  EQUITY_FUND   
5       11:49:44     2020-04-03      GBP  EQUITY_FUND   
6       11:49:44     2020-04-03      GBP  EQUITY_FUND   
7       11:49:44     2020-04-03      GBP  EQUITY_FUND   
8       11:49:44     2020-04-03      GBP  EQUITY_FUND   
9       11:49:44     2020-04-03      GBP  EQUITY_FUND   
10      11:49:44     2020-04-03      GBP  EQUITY_FUND   
11      11:49:44     2020-04-03      GBP  EQUITY_FUND   
12      11:49:44     2020-04-04      GBP  EQUITY_FUND   
13      11:49:44     2020-04-04      GBP  EQUITY_FUND   
14      11:49:44     2020-04-04      GBP  EQUITY_FUND   
15      11:49:44     2020-04-05      GBP  EQUITY_FUND   
16   

# Upserting Recipe

We need to upsert a recipe to be able to use upserted quotes -> dependency

In [117]:
from lusid.api import ConfigurationRecipeApi
from lusid.models import configuration_recipe, market_context, market_data_key_rule, UpsertRecipeRequest, ResourceId
configuration_api: ConfigurationRecipeApi = api_factory.build(ConfigurationRecipeApi)
recipe_id =  ResourceId(scope, "recipe-code-001")
recipe_response = configuration_api.upsert_configuration_recipe(UpsertRecipeRequest(
  configuration_recipe = configuration_recipe.ConfigurationRecipe(
    recipe_id.scope, 
    recipe_id.code, 
    market = market_context.MarketContext([
      market_data_key_rule.MarketDataKeyRule("Quote.*.*", "Lusid", scope, "Price", "mid", "30D"),
      market_data_key_rule.MarketDataKeyRule("Quote.ClientInternal.*", "Lusid", scope, "Price", "mid", "30D"),
      # market_data_key_rule.MarketDataKeyRule("Rates.*.*", "Lusid", scope, "Price", "mid", "30D") 
    ]))))
print(recipe_response)


{'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://fbn-ci.lusid.com/app/insights/logs/0HMO8VVRKP5OH:0000007E',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'value': datetime.datetime(2023, 2, 7, 11, 49, 44, 499672, tzinfo=tzutc())}


# Query raw cashflows
Note: This will return an empty array due to not having any instrument cashflows only fund movements

In [136]:
from lusid.api import TransactionPortfoliosApi
from lusid.models import QueryCashFlowsRequest, PortfolioEntityId
transaction_portfolio_api: TransactionPortfoliosApi = api_factory.build(TransactionPortfoliosApi)
dates = pd.to_datetime(transactions_df["txn_trade_date"]).map(lambda x: x.replace(day = 1).tz_localize('UTC')).map(lambda x: pd.Timestamp.isoformat(x))

cashflow_response = transaction_portfolio_api.get_portfolio_cash_flows(
    scope, 
    portfolio_code,
    recipe_id_scope= recipe_id.scope,
    recipe_id_code= recipe_id.code,
)
print(cashflow_response)

{'href': 'https://fbn-ci.lusid.com/api/api/transactionportfolios/cash-ladder-001/EQUITY_FUND/cashflows/01%2F01%2F0001%2000%3A00%3A00%20%2B00%3A0012%2F31%2F9999%2023%3A59%3A59%20%2B00%3A00?recipeIdCode=recipe-code-001&recipeIdScope=cash-ladder-001',
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://fbn-ci.lusid.com/app/insights/logs/0HMO907Q83VBV:00000004',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'next_page': None,
 'previous_page': None,
 'values': []}


# Cashflow bucketing
- Note: We only have fund movements and no instrument cash flows
- Note: This will return an empty due to default bucketing only being applied on instrument cashflows.

In [123]:
from lusid.api import TransactionPortfoliosApi, QuotesApi
from lusid.models import UpsertQuoteRequest, QuoteId, QuoteSeriesId, MetricValue, BucketedCashFlowRequest, ResourceId
transaction_portfolio_api: TransactionPortfoliosApi = api_factory.build(TransactionPortfoliosApi)
quotes_api: QuotesApi = api_factory.build(QuotesApi)

def create_quote_request(instrument_id: str, instrument_id_type: str, value: float, currency: str, field: str = "mid", effective_at: datetime = datetime.now(pytz.utc)):
    return UpsertQuoteRequest(
            quote_id=QuoteId(
                quote_series_id=QuoteSeriesId(
                    provider="Lusid",
                    instrument_id=instrument_id, # row["client_internal"],
                    instrument_id_type=instrument_id_type, # "ClientInternal",
                    quote_type="Price",
                    field=field,
                ),
                effective_at=effective_at.isoformat(),
            ),
            metric_value=MetricValue(value=value, unit=currency),
            scale_factor=100,
        )

# print(transactions_df["txn_trade_date"])
instruments = transactions_df.loc[transactions_df["instrument_type"] != "cash", ["instrument_id", "currency", "instrument_type"]].drop_duplicates(["instrument_id"]) # get non cash instruments
print(instruments)

quote_reqs: pd.DataFrame = instruments.apply(lambda row: create_quote_request(row["instrument_id"], "ClientInternal", 1000, row["currency"]), axis=1)
# print(quote_reqs.to_dict())
# print(len(quote_reqs.to_dict()))
print("upsert quotes to scope: ", scope)
quote_upsert_response = quotes_api.upsert_quotes(scope, request_body = quote_reqs.to_dict())
print(quote_upsert_response.failed)
    


bucketing_dates = pd.to_datetime(transactions_df["txn_trade_date"]).map(lambda x: x.replace(day = 1).tz_localize('UTC'))
print(bucketing_dates.map(lambda x: pd.Timestamp.isoformat(x)).unique())

bucketing_response = transaction_portfolio_api.get_bucketed_cash_flows(
    scope = scope,
    code = portfolio_code,
    bucketed_cash_flow_request= BucketedCashFlowRequest(
        "RoundDown",
        bucketing_dates.map(lambda x: pd.Timestamp.isoformat(x)).unique().tolist(),
        None,
        recipe_id= recipe_id
    )
)
print(bucketing_response.failed)
print(bucketing_response.data)

   instrument_id currency instrument_type
0       EQ_GB001      GBP          equity
3       EQ_GB002      GBP          equity
5       EQ_GB003      GBP          equity
6       EQ_GB004      GBP          equity
13      EQ_US001      USD          equity
14      EQ_US002      USD          equity
15      EQ_US003      USD          equity
16      EQ_US004      USD          equity
upsert quotes to scope:  cash-ladder-001
{}
['2020-01-01T00:00:00+00:00' '2020-03-01T00:00:00+00:00']
{}
[]
